# Cornell Birdcall Identification 🐦
![](https://www.depthworld.com/wp-content/uploads/2019/03/Quetzal.jpg)

Most birds have a wide repertoire of songs and call, but there’s an important distinction to be made between the two. Among the songbirds and various other groups of birds (such as cuckoos, owls, and nightjars), songs are used to defend territory and attract mates. Therefore, it’s the males that sing the most—usually during breeding season.

## Upvote if you like my kernel

### 📎 Types of Calls

* Alarm calls

* Contact calls

* Flight calls

* Begging calls

* Describing Sounds

* Trill

* Buzz and Thin

* Harsh

* Bell-like, flute-like, whistling, or metallic

* Other Noises
There’s more to bird sounds than just vocals. Downy Woodpeckers advertise their presence by drumming rapidly on a tree—and sometimes on the side of your house. (In fact, you can ID certain woodpecker species by measuring the pace of their knocks.) Mating season is often full of unexpected noises like the soft thwacks of a Ruffed Grouse’s wings or the squeaky vibrato of an American Woodcock’s flight feathers. Listen for subtler sounds, too, such as shuffling leaves, flapping feathers, and clumsy, crashing fowl in water. They can point to behavioral clues and help you solve an ID. 

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import xgboost
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

%matplotlib inline
ds=pd.read_csv("../input/birdsong-recognition/train.csv")

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = ds['longitude'],
        lat = ds['latitude'],
        text = ds['location'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = ds['duration'],
            cmax = ds['duration'].max(),
            colorbar_title="Duration"
        )))

fig.update_layout(
        title = 'Recorded Place all over the world',
        geo = dict(
            scope='world',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
fig.show()

In [ ]:
val=ds["pitch"].value_counts()
nt=["Not specified"]

lvl=["level"] 
bt=["both"]
dc=["decreasing"]
inc=["increasing"]

fig = go.Figure(data=[
    go.Bar(name='Not specified', x=nt, y=[val["Not specified"]]),
    go.Bar(name='Level', x=lvl, y=[val["level"]]),
    go.Bar(name='Both', x=bt, y=[val["both"]]),
    go.Bar(name='decreasing', x=dc, y=[val["decreasing"]]),
    go.Bar(name='increasing', x=inc, y=[val["increasing"]])
    
    
])
# Change the bar mode
fig.update_layout(barmode='group',title="Pitch Labels")
fig.show()

In [ ]:
import librosa
audio_path = '../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3'
x , sr = librosa.load(audio_path)
print(type(x), type(sr))
print(x.shape, sr)


In [ ]:
librosa.load(audio_path, sr=None)


# 📎 Waveform 🔊 

#### A waveform is an image that represents an audio signal or recording. It shows the changes in amplitude over a certain amount of time. The amplitude of the signal is measured on the y-axis (vertically), while time is measured on the x-axis (horizontally).


In [ ]:
import IPython.display as ipd
ipd.Audio(audio_path)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

### plot of the amplitude envelope of a waveform.

# 📎 Spectrogram


In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

#### A spectrogram is a visual representation of the spectrum of frequencies of sound or other signals as they vary with time. Spectrograms are sometimes called sonographs, voiceprints, or voicegrams. When the data is represented in a 3D plot, they may be called waterfalls. In 2-dimensional arrays, the first axis is frequency while the second axis is time.

In [ ]:
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

### vertical axis shows frequencies (from 0 to 10kHz), and the horizontal axis shows the time of the clip

# 📎 Zero Crossing Rate

The zero-crossing rate is the rate of sign-changes along a signal, i.e., the rate at which the signal changes from positive to zero to negative or from negative to zero to positive. This feature has been used heavily in both speech recognition and music information retrieval, being a key feature to classify percussive sounds.

In [ ]:
x, sr = librosa.load('../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3')
#Plot the signal:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
n0 = 9000
n1 = 9100
plt.figure(figsize=(14, 5))
plt.plot(x[n0:n1])
plt.grid()

# 📎 Spectral Centroid

### The spectral centroid is a measure used in digital signal processing to characterise a spectrum. It indicates where the center of mass of the spectrum is located. Perceptually, it has a robust connection with the impression of brightness of a sound.

In [ ]:
import sklearn
spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
spectral_centroids.shape
(775,)
# Computing the time variable for visualization
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)
# Normalising the spectral centroid for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#Plotting the Spectral Centroid along the waveform
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='r')

# 📎 Spectral Rolloff
### Spectral rolloff is the frequency below which a specified percentage of the total spectral energy, e.g. 85%, lies.

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='r')

# 📎 Mel-Frequency Cepstral Coefficients

### In sound processing, the mel-frequency cepstrum is a representation of the short-term power spectrum of a sound, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency. Mel-frequency cepstral coefficients are coefficients that collectively make up an MFC.

In [ ]:
x, fs = librosa.load('../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3')
librosa.display.waveplot(x, sr=sr)

In [ ]:
mfccs = librosa.feature.mfcc(x, sr=fs)
print(mfccs.shape)
(20, 97)
#Displaying  the MFCCs:
librosa.display.specshow(mfccs, sr=sr, x_axis='time')

In [ ]:
import sklearn
mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
print(mfccs.mean(axis=1))
print(mfccs.var(axis=1))
librosa.display.specshow(mfccs, sr=sr, x_axis='time')

# 📎 Chroma Frequencies

### Chroma features are an interesting and powerful representation for music audio in which the entire spectrum is projected onto 12 bins representing the 12 distinct semitones (or chroma) of the musical octave

In [ ]:
# Loadign the file
x, sr = librosa.load('../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3')
hop_length = 512
chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')

### Would Continue....